In [ ]:
import pandas as pd

df = pd.read_excel(r"C:/Users/HP/Desktop/Jupiter/Datos_Depurados2.xlsx")

# =========================
# Limpiar PIB per cápita y convertir a número (robusto)
# =========================
df['PIB per capita'] = df['PIB per capita'].astype(str)
df['PIB per capita'] = df['PIB per capita'].str.replace(r'[^\d.]','', regex=True)
df['PIB per capita'] = pd.to_numeric(df['PIB per capita'], errors='coerce')
df = df.dropna(subset=['PIB per capita'])

# =========================
# Convertir Año a número
# =========================
df['Año'] = pd.to_numeric(df['Año'], errors='coerce')


# =========================
# Unificar regiones → continentes
# =========================
mapa_continentes = {
    'Northern/Eastern Europe': 'Europa',
    'European Union': 'Europa',
    'EU-27': 'Europa',
    'Other Europe': 'Europa',

    'Latin America': 'América',
    'North America': 'América',
    'Central America/Caribbean': 'América',
    'South America': 'América',

    'Middle East/Central Asia': 'Asia',
    'Asia-Pacific': 'Asia',

    'Africa': 'África',
    'Sub-Saharan Africa': 'África',

    'Oceania': 'Oceanía',
    'oceania': 'Oceanía',
    'ocenia': 'Oceanía'
}

df['Continente'] = df['Region'].map(mapa_continentes)

# Verificación rápida
print("Regiones sin continente:", df['Continente'].isna().sum())

# =========================
# Función Banco Mundial
# =========================
def clasificar_ingreso_bm(pib, año):
    if año == 2016:
        if pib <= 1025:
            return 'Ingreso bajo'
        elif pib <= 12475:
            return 'Ingreso medio'
        else:
            return 'Ingreso alto'
    elif año == 2023:
        if pib <= 1135:
            return 'Ingreso bajo'
        elif pib <= 13845:
            return 'Ingreso medio'
        else:
            return 'Ingreso alto'

# =========================
# Clasificación por ingreso
# =========================
df['Nivel de ingreso'] = df.apply(
    lambda fila: clasificar_ingreso_bm(
        fila['PIB per capita'],
        fila['Año']
    ),
    axis=1
)

# =========================
# Crear tabla pivot con listas de países por continente y nivel de ingreso
# =========================
def pivot_paises(df_anyo):
    tabla = (
        df_anyo
        .groupby(['Continente', 'Nivel de ingreso'])
        .agg({'Pais': lambda x: ', '.join(x)})  # unir países en un string
        .reset_index()
    )
    # Pivotear: filas = Continente, columnas = Nivel de ingreso
    tabla_pivot = tabla.pivot(index='Continente', columns='Nivel de ingreso', values='Pais')
    tabla_pivot = tabla_pivot.reset_index()  # volver a poner 'Continente' como columna
    return tabla_pivot

paises_2016_pivot = pivot_paises(df[df['Año'] == 2016])
paises_2023_pivot = pivot_paises(df[df['Año'] == 2023])

# =========================
# Guardar en Excel
# =========================
with pd.ExcelWriter(r"C:/Users/HP/Desktop/Jupiter/Tablas_Continente_PIB_BM.xlsx") as writer:
    # Conteo de países
    tabla_2016.to_excel(writer, sheet_name='2016', index=False)
    tabla_2023.to_excel(writer, sheet_name='2023', index=False)
    
    # Listas de países en formato pivot “ancho”
    paises_2016_pivot.to_excel(writer, sheet_name='Países 2016', index=False)
    paises_2023_pivot.to_excel(writer, sheet_name='Países 2023', index=False)



print("\n✔ Tablas creadas correctamente")